# Transforming Unstructured Text into Structured Data 
Online supplementary material to "The Evolving U.S. Occupational Structure: A Textual Analysis." 

* [Most recent version of the paper](
http://ssc.wisc.edu/~eatalay/skills.pdf)

* [Project data library](http://ssc.wisc.edu/~eatalay/occupation_data.html) 

* [GitHub repository](https://github.com/phaiptt125/newspaper_project)

***

This IPython notebook demonstrates how we finally transform unstructured newspaper text into structured data (spreadsheet). In the previous steps, we:

* Retrieve document metadata, remove markup from the newspaper text, and to perform an initial spell-check of the text (see [here](https://github.com/phaiptt125/newspaper_project/blob/master/data_cleaning/initial_cleaning.ipynb)). 
* Exclude non-job ad pages (see [here](https://github.com/phaiptt125/newspaper_project/blob/master/data_cleaning/LDA.ipynb)).

The main components of this step are to identify the job title, discern the boundaries between job ads and transform relevant information into structured data. 



<b> Due to copyright restrictions, we are not authorized to publish a large body of newspaper text. </b>
***

### List of auxiliary files (see project data library or GitHub repository)
* *title_detection.py* :
* *title_substitute* :
* *compute_spelling* :

***
## Import necessary modules

In [1]:
import os
import re
import sys
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
 
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

sys.path.append('./auxiliary files')

from compute_spelling import *
from title_substitute import *
from title_detection import *
from detect_ending import *

## Import Job Ad Pages

We present an example describing how our procedure identifies job ads' boundaries and their job titles on a snippet of Display Ad page 226, from the January 14, 1979 Boston Globe (page identifer: "Globe_displayad_19790114_226"). 

* The text file has already been cleaned by retrieving document metadata, removing markup from the newspaper text, and correcting spelling error of the text (see [here](https://github.com/phaiptt125/newspaper_project/blob/master/data_cleaning/initial_cleaning.ipynb)). 
* We have already classified this page to be related to job ads (see [here](https://github.com/phaiptt125/newspaper_project/blob/master/data_cleaning/LDA.ipynb)).

In [2]:
text = open('Snippet_Globe_displayad_19790114_226.txt').read()
page_identifier = 'Globe_displayad_19790114_226'
print(text)

MEDICAL HELP
NUCLEAR
RADIOLOGIC TECH
full time day po ition is available for registred or registry technician in our Nuclear Medicine department This position does require taking call
CHEST
PHYSICAL THERAPIST
If you are or registry eligible
Physical Trhrapist interested in Chest
Therapy consider the New England Baptist Hospital Responsibilities will include providng chest therapy for Medical Surgical patients family teaching interdisciplinary inservice programs and more
For more information please contact our Personnel department 738-5800 , Ext 255 . An Equal Opportunity Employer
41 Pa HII Boston
MANAGER OF
PRIMARY CARE PROGRAMS
Children's Hospital Medical Center
seeks dynamic creative individual to manage its Primary Care Programs including 24-hour Emergency Room Primary Care program the Massachusetts Poison information Center and
Dental services This position requires 3-5 years experience with background in planning budgeting and managing
health programs Masters degree preferred but 

## Reset Line Breaks

In [3]:
# remove emypty lines
text_by_line = [w for w in re.split('\n',text) if not w=='']

# reset lines
text_reset_line = CombineUppercase(text_by_line)
text_reset_line = UppercaseNewline(text_reset_line,'\n') #assign new line when an uppercase word is found
text_reset_line = CombineUppercase(text_reset_line) #re-combine uppercase words together

# remove extra white spaces
text_reset_line = [' '.join([y for y in re.split(' ',w) if not y=='']) for w in text_reset_line]
# remove empty lines
text_reset_line = [w for w in text_reset_line if not w=='']

In [4]:
# print results
text_reset_line

['MEDICAL HELP NUCLEAR RADIOLOGIC TECH',
 'full time day po ition is available for registred or registry technician in our Nuclear Medicine department This position does require taking call',
 'CHEST PHYSICAL THERAPIST',
 'If you are or registry eligible',
 'Physical Trhrapist interested in Chest',
 'Therapy consider the New England Baptist Hospital Responsibilities will include providng chest therapy for Medical Surgical patients family teaching interdisciplinary inservice programs and more',
 'For more information please contact our Personnel department 738-5800 , Ext 255 . An Equal Opportunity Employer',
 '41 Pa',
 'HII',
 'Boston',
 'MANAGER OF PRIMARY CARE PROGRAMS',
 "Children's Hospital Medical Center",
 'seeks dynamic creative individual to manage its Primary Care Programs including 24-hour Emergency Room Primary Care program the Massachusetts Poison information Center and',
 'Dental services This position requires 3-5 years experience with background in planning budgeting and 

## Detect Job Titles

In [5]:
# define indicators if title detected
title_found = '---titlefound---'

# read all input files
TitleBaseFile = open('./auxiliary files/TitleBase.txt').read()
TitleBaseList = [w for w in re.split('\n',TitleBaseFile) if not w=='']

In [6]:
text_detect_title = ['']*len(text_reset_line)
PreviousLineIsUppercaseTitle = False

for i in range(0,len(text_reset_line)):
    line = text_reset_line[i]
    line_no_hyphen = re.sub('-',' ',line.lower())
    tokens = word_tokenize(line_no_hyphen)
    
    Match = list(set(tokens).intersection(TitleBaseList)) # see if the line has words in TitleBaseList 
        
    if Match and DetermineUppercase(line): # uppercase job title
        text_detect_title[i] = ' '.join([w for w in re.split(' ',line) if not w=='']) + title_found
        # adding a flag that a title is found
        # ' '.join([w for w in split(' ',line) if not w=='']) is to remove extra spaces from 'line'
        PreviousLineIsUppercaseTitle = True
    elif Match and len(tokens) <= 3:
        # This line allows non-uppercase job titles
        # It has to be short enough => less than or equal to 3 words
        # To be more conservative, the previous line must NOT be a uppercase job title 
        if PreviousLineIsUppercaseTitle == False:
            text_detect_title[i] = ' '.join([w for w in re.split(' ',line) if not w=='']) + title_found
            PreviousLineIsUppercaseTitle = False
        else:
            text_detect_title[i] = ' '.join([w for w in re.split(' ',line) if not w==''])
            PreviousLineIsUppercaseTitle = False
    else:
        text_detect_title[i] = ' '.join([w for w in re.split(' ',line) if not w==''])
        PreviousLineIsUppercaseTitle = False

In [7]:
[w for w in text_detect_title if re.findall(title_found,w)]

['MEDICAL HELP NUCLEAR RADIOLOGIC TECH---titlefound---',
 'CHEST PHYSICAL THERAPIST---titlefound---',
 'MANAGER OF PRIMARY CARE PROGRAMS---titlefound---',
 'MEDICAL---titlefound---',
 'MF MEDICAL GROUP MANAGER---titlefound---',
 'CAPE ANN MEDICAL CENTER---titlefound---',
 'RN---titlefound---',
 'APPLY PERSONNEL DEPARTMENT CAPE COD HOSPITAL---titlefound---',
 'MEDICAL RECORDS SUPERVISOR---titlefound---',
 'MEDICAL RECEPTIONIST---titlefound---']

## Detect Addresses and Endings Phrases 

In [8]:
ending_found = '---endingfound---'
text_assign_flag = list()

for line in text_detect_title:
    AddressFound , EndingPhraseFound = AssignFlag(line)
    if AddressFound == True or EndingPhraseFound == True:
        text_assign_flag.append(line+ending_found)
    else:
        text_assign_flag.append(line)

In [9]:
[w for w in text_assign_flag if re.findall(ending_found,w)]

['For more information please contact our Personnel department 738-5800 , Ext 255 . An Equal Opportunity Employer---endingfound---',
 '300 Lonjwood Avenue---endingfound---',
 'For further information please contact the Personrel department---endingfound---',
 '580 Court Street Keene NH 03431---endingfound---',
 'Gloucester Mass 01930---endingfound---',
 'Hyannis MA 02601---endingfound---',
 'please send resume qnd requirements in confidence to Director of Personnel---endingfound---',
 'Hospital Road Malden Mass 02148---endingfound---',
 'Synd resume to Dobra Kiley -Davis 50 Binney Strett Boston MA 02115---endingfound---']

In [10]:
text_assign_flag

['MEDICAL HELP NUCLEAR RADIOLOGIC TECH---titlefound---',
 'full time day po ition is available for registred or registry technician in our Nuclear Medicine department This position does require taking call',
 'CHEST PHYSICAL THERAPIST---titlefound---',
 'If you are or registry eligible',
 'Physical Trhrapist interested in Chest',
 'Therapy consider the New England Baptist Hospital Responsibilities will include providng chest therapy for Medical Surgical patients family teaching interdisciplinary inservice programs and more',
 'For more information please contact our Personnel department 738-5800 , Ext 255 . An Equal Opportunity Employer---endingfound---',
 '41 Pa',
 'HII',
 'Boston',
 'MANAGER OF PRIMARY CARE PROGRAMS---titlefound---',
 "Children's Hospital Medical Center",
 'seeks dynamic creative individual to manage its Primary Care Programs including 24-hour Emergency Room Primary Care program the Massachusetts Poison information Center and',
 'Dental services This position require

## Assign Boundaries

In [11]:
split_indicator = '---splithere---'

# -----Step1: split if title is found-----

split_by_title = list() 

for line in text_assign_flag:
    if re.findall(title_found,line):
        #add a split indicator BEFORE the line with title 
        split_by_title.append(split_indicator + '\n' + line)
    else:
        split_by_title.append(line) # if not found, just append the line back in 

    # "SplitAdTitleByLine" is supposed to be a list where each element is one line.
    # However, there'd be extra lines due to split indicator (where a title is found).
    # The command below first joins all the lines together '\n'.join() and then
    # split again by '\n' so that each element indeed represents one line
            
split_by_title = [w for w in re.split('\n','\n'.join(split_by_title)) if not w=='']

In [12]:
split_posting = list()

# -----Step2: split if any ending phrase and/or address is found-----

for line in split_by_title:
    line_remove_ending_found = re.sub(ending_found,'',line)
    if re.findall(ending_found,line):
        #add a split indicator AFTER the line where the pattern is found
        split_posting.append( line_remove_ending_found + '\n' + split_indicator)
    else:
        split_posting.append( line_remove_ending_found ) # if not found, just append the line back in 
            
split_posting = [w for w in re.split(split_indicator,'\n'.join(split_posting)) if not w=='']

In [16]:
for ad in split_posting:
    print(ad)
    print('---splithere---')


MEDICAL HELP NUCLEAR RADIOLOGIC TECH---titlefound---
full time day po ition is available for registred or registry technician in our Nuclear Medicine department This position does require taking call

---splithere---

CHEST PHYSICAL THERAPIST---titlefound---
If you are or registry eligible
Physical Trhrapist interested in Chest
Therapy consider the New England Baptist Hospital Responsibilities will include providng chest therapy for Medical Surgical patients family teaching interdisciplinary inservice programs and more
For more information please contact our Personnel department 738-5800 , Ext 255 . An Equal Opportunity Employer

---splithere---

41 Pa
HII
Boston

---splithere---

MANAGER OF PRIMARY CARE PROGRAMS---titlefound---
Children's Hospital Medical Center
seeks dynamic creative individual to manage its Primary Care Programs including 24-hour Emergency Room Primary Care program the Massachusetts Poison information Center and
Dental services This position requires 3-5 years expe

## Create Dataset in a Spreadsheet Format

In [14]:
all_flag = re.compile('|'.join([title_found,ending_found]))

num_ad = 0 #initialize ad number within displayad

final_output = list()

for ad in split_posting:
    
    ad_split_line = [w for w in re.split('\n',ad) if not w=='']
        
    # --------- record title ----------

    title_this_ad = [w for w in ad_split_line if re.findall(title_found,w)] 
    #see if any line is a title
            
    if len(title_this_ad) == 1: #if we do have a title
        title_clean = re.sub(all_flag,'',title_this_ad[0].lower()) 
        #take out the flags and revert to lowercase

        title_clean = ' '.join([y for y in re.split(' ',title_clean) if not y==''])
    else:
        title_clean = ''

    # --------- record content ----------
        
    ad_content = [w for w in ad_split_line if not re.findall(title_found,w)] # take out lines with title
    ad_content = ' '.join([w for w in ad_content if not w==''])
    #delete empty lines + combine all the line together (within an ad)
        
    ad_content = re.sub(all_flag,'',ad_content.lower()) 
    #take out all the flags

    # --------- record output ----------

    num_ad += 1
    output = [str(page_identifier),str(num_ad),str(title_clean),str(ad_content)]    
    final_output.append( '|'.join(output) )

final_output_file = open('structured_posting.txt','w')
final_output_file.write('\n'.join(final_output))
final_output_file.close()

In [19]:
structured_posting = open('structured_posting.txt').read()
structured_posting = re.split('\n',structured_posting)
for ad in structured_posting:
    print(ad)

Globe_displayad_19790114_226|1|medical help nuclear radiologic tech|full time day po ition is available for registred or registry technician in our nuclear medicine department this position does require taking call
Globe_displayad_19790114_226|2|chest physical therapist|if you are or registry eligible physical trhrapist interested in chest therapy consider the new england baptist hospital responsibilities will include providng chest therapy for medical surgical patients family teaching interdisciplinary inservice programs and more for more information please contact our personnel department 738-5800 , ext 255 . an equal opportunity employer
Globe_displayad_19790114_226|3||41 pa hii boston
Globe_displayad_19790114_226|4|manager of primary care programs|children's hospital medical center seeks dynamic creative individual to manage its primary care programs including 24-hour emergency room primary care program the massachusetts poison information center and dental services this position r